# Import Lib
Neural machine translation with a Transformer and Keras - https://www.tensorflow.org/text/tutorials/transformer#the_transformer

In [ ]:
#!pip install tensorflow_text

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [15,8]
import seaborn as sns
import plotly.graph_objects as go

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Embedding, Layer, MultiHeadAttention, LayerNormalization, Add, Dense, Dropout, Flatten
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow.math import rsqrt, minimum
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.ops import round as tf_round

In [ ]:
from tensorflow.signal import stft, hann_window

In [ ]:
import math
import joblib

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error

In [ ]:
from tensorflow.keras import initializers

initializer_for_relu = initializers.HeNormal() # For layers with activation function Relu
initializer_for_sigmoid = initializers.GlorotNormal() # For layers with activation function Sigmoid

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian

In [ ]:
from tensorflow.python.framework import tensor_util

# Data Preprocessing

## Import Data

In [ ]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/hobby_project/Data'
data_df = pd.read_csv(path+'/raw_nsepy_inp512_differencedVal_fourierTransform.csv')
data_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,512
0,-4.45,6.45,-3.1,3.00,3.10,-2.55,-3.25,9.45,1.55,-5.2,...,-3.25,5.70,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35
1,6.45,-3.10,3.0,3.10,-2.55,-3.25,9.45,1.55,-5.20,-4.0,...,5.70,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35,2.05
2,-3.10,3.00,3.1,-2.55,-3.25,9.45,1.55,-5.20,-4.00,1.5,...,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35,2.05,1.70


In [ ]:
data_df.shape

(41412, 513)

## Train Test Split
* Length of the input 39 <br>
* Length of Output and Context 1 <br>

In [ ]:
data_df.iloc[:600,256:].shape

(600, 257)

In [ ]:
data_df.iloc[:600,256:].head(3)

,256,257,258,259,260,261,262,263,264,265,...,503,504,505,506,507,508,509,510,511,512
0,2.75,-0.40,-1.65,-0.85,-1.0,0.30,3.90,7.85,2.75,4.30,...,-3.25,5.70,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35
1,-0.40,-1.65,-0.85,-1.00,0.3,3.90,7.85,2.75,4.30,-5.35,...,5.70,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35,2.05
2,-1.65,-0.85,-1.00,0.30,3.9,7.85,2.75,4.30,-5.35,-0.25,...,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35,2.05,1.70


In [ ]:
data_q1 = data_df.iloc[:600,:129].copy()
data_q2 = data_df.iloc[:600,128:257].copy()
data_q3 = data_df.iloc[:600,256:385].copy()
data_q4 = data_df.iloc[:600,384:].copy()
data_df_merged = pd.DataFrame()
data_df_merged = pd.concat([pd.DataFrame(data_q1.values),pd.DataFrame(data_q2.values),pd.DataFrame(data_q3.values),pd.DataFrame(data_q4.values)],
                           ignore_index=True)
print(data_df_merged.shape)
data_df_merged.head(3)

(2400, 129)


,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,-4.45,6.45,-3.1,3.00,3.10,-2.55,-3.25,9.45,1.55,-5.2,...,-7.65,-1.25,0.00,8.00,-6.05,1.80,-0.75,-0.9,-0.1,-2.8
1,6.45,-3.10,3.0,3.10,-2.55,-3.25,9.45,1.55,-5.20,-4.0,...,-1.25,0.00,8.00,-6.05,1.80,-0.75,-0.90,-0.1,-2.8,-1.5
2,-3.10,3.00,3.1,-2.55,-3.25,9.45,1.55,-5.20,-4.00,1.5,...,0.00,8.00,-6.05,1.80,-0.75,-0.90,-0.10,-2.8,-1.5,0.5


In [ ]:
data_df_merged.tail(3)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
2397,-19.25,4.05,-2.15,-3.80,-2.20,7.85,5.95,-3.65,0.90,1.00,...,-5.35,-0.70,9.90,-2.05,4.95,-2.55,-0.55,1.80,-2.30,1.95
2398,4.05,-2.15,-3.80,-2.20,7.85,5.95,-3.65,0.90,1.00,8.75,...,-0.70,9.90,-2.05,4.95,-2.55,-0.55,1.80,-2.30,1.95,3.85
2399,-2.15,-3.80,-2.20,7.85,5.95,-3.65,0.90,1.00,8.75,-1.50,...,9.90,-2.05,4.95,-2.55,-0.55,1.80,-2.30,1.95,3.85,2.55


In [ ]:
inp_len = 128 # Length of the input
out_len = 1 # Length of Output and Context

In [ ]:
prediction_percentage = 0.1 #0.025
#train_df, test_df = train_test_split(data_df, test_size=prediction_percentage, random_state=1, shuffle=True)
train_df, test_df = train_test_split(data_df_merged, test_size=prediction_percentage, random_state=1, shuffle=True)

In [ ]:
print('Train shape',train_df.shape)
print('Test shape',test_df.shape)

Train shape (2160, 129)
Test shape (240, 129)


In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=1, shuffle=True)

In [ ]:
print('Train shape',train_df.shape)
print('Validation shape',val_df.shape)
print('Test shape',test_df.shape)

Train shape (1620, 129)
Validation shape (540, 129)
Test shape (240, 129)


Keras Model.fit training expects (inputs, labels) pairs. The inputs is tokenized sequences. The labels are the same sequences shifted by 1. This shift is so that at each location input sequence, the label in the next token.

In [ ]:
X_train = train_df.drop(columns=train_df.columns[inp_len:].values).values
c_train = train_df.drop(columns=train_df.columns[:inp_len].values).values
y_train = c_train
X_val = val_df.drop(columns=val_df.columns[inp_len:].values).values
c_val = val_df.drop(columns=val_df.columns[:inp_len].values).values
y_val = c_val
X_test = test_df.drop(columns=test_df.columns[inp_len:].values).values
c_test = test_df.drop(columns=test_df.columns[:inp_len].values).values
y_test = c_test

In [ ]:
print('X Train shape',X_train.shape)
print('Context Train shape',c_train.shape)
print('Y Train shape',y_train.shape)
print('X Validation shape',X_val.shape)
print('Context Validation shape',c_val.shape)
print('Y Validation shape',y_val.shape)
print('X Test shape',X_test.shape)
print('Context Test shape',c_test.shape)
print('Y Test shape',y_test.shape)

X Train shape (1620, 128)
Context Train shape (1620, 1)
Y Train shape (1620, 1)
X Validation shape (540, 128)
Context Validation shape (540, 1)
Y Validation shape (540, 1)
X Test shape (240, 128)
Context Test shape (240, 1)
Y Test shape (240, 1)


In [ ]:
X_train_f = X_train.astype(np.float32)
y_train_t = tf.convert_to_tensor(y_train)
X_val_f = X_val.astype(np.float32)
y_val_t = tf.convert_to_tensor(y_val)
X_test_f = X_test.astype(np.float32)
y_test_t = tf.convert_to_tensor(y_test)

In [ ]:
X_train_f[0]

array([  2.75,  -4.4 ,  -0.95,  -1.3 ,  -1.5 ,   6.8 ,  -2.  ,   4.85,
         2.55,  -1.15,   8.95,   0.4 ,   1.45,  -5.05,   1.3 ,   6.7 ,
         2.  ,   4.15,  -5.8 ,  -4.75,   2.55,   0.35,  -1.25,   1.25,
        -1.4 ,   9.  ,  -1.7 ,  -3.2 ,   2.55,   0.7 , -11.65,   2.7 ,
         5.7 ,  -2.8 ,   1.25,  -3.65,   1.6 ,  -0.6 ,   1.4 ,   0.9 ,
         3.7 ,  -2.25,  -0.8 ,   0.3 ,  -2.7 ,   1.45,   1.6 ,   0.35,
         1.2 ,  -2.9 ,  -0.25,   1.1 ,   6.8 ,   2.5 ,  -2.9 ,   4.5 ,
        -0.5 ,  -5.5 ,   5.  ,  -2.15,  -0.85,  -2.  ,  -0.95,   2.95,
        -3.4 ,  -0.55,   0.15,  -3.25,  -4.95,  -3.4 ,  -0.35,  -1.45,
        -1.1 ,  -1.5 ,   4.15,  -4.35,  -1.45,   4.9 ,  10.3 ,   2.95,
         5.25,  -5.45,   3.5 ,   1.15,  -4.15,  -1.7 ,   3.15,   0.75,
        -1.95,   0.75,   4.  , -11.  ,   4.5 ,   0.5 ,   0.75,  -1.1 ,
         4.2 ,  -0.75,   2.45,   0.55,   9.9 ,   3.95,   1.4 ,   4.6 ,
        -2.95,  -0.5 ,  -3.25,   5.7 ,  -5.45,   3.2 ,  -4.45,  -2.15,
      

In [ ]:
y_train[0]

array([-0.6])

In [ ]:
y_train_t[0]

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-0.6])>

## Testing tf.signal.stft

In [ ]:
spectrogram1 = tf.signal.stft(signals=X_train_f[0],
                              frame_length=512,
                              frame_step=1)

print(X_train[0].shape[0])
print(spectrogram1.shape)
print(spectrogram1)

512
(1, 257)
tf.Tensor(
[[ 111.96902     +0.j         -15.119688  -159.64772j
   -72.89093   +208.4931j      69.34505   -213.79272j
     0.42047882+146.51797j    -52.74468     -1.3395691j
   -41.004944   -47.943695j    25.380985   -65.980995j
    38.980003   +92.98576j     97.73398    +63.70668j
  -287.45038   -158.26791j    321.8197    +151.30878j
  -136.15845   -139.39746j    -22.614635   +88.557274j
   -75.58975    +16.577293j    75.36231    -14.467594j
    75.512596  -103.24147j    -65.76552    +48.333424j
    51.243362   +15.50803j    -77.67269    +78.130295j
    36.67746    -47.848145j    -0.8291626  -37.583656j
    54.28049    +15.14283j    -69.24906    +49.755505j
    17.982956  -137.15686j      8.250641  +204.92694j
   -84.98456   -216.10516j    133.65607   +204.3157j
  -108.657455  -182.85588j    156.60089   +116.13028j
  -201.24722    -19.4309j      75.85985    -19.267372j
    83.98462    -15.202744j   -22.855158   +40.50888j
  -153.69621    +14.0824585j  111.61706    -98.27

In [ ]:
spectrogram2 = tf.signal.stft(X_train_f[0], frame_length=512, frame_step=256)

print(X_train[0].shape[0])
print(spectrogram2.shape)
print(spectrogram2)

512
(1, 257)
tf.Tensor(
[[ 111.96902     +0.j         -15.119688  -159.64772j
   -72.89093   +208.4931j      69.34505   -213.79272j
     0.42047882+146.51797j    -52.74468     -1.3395691j
   -41.004944   -47.943695j    25.380985   -65.980995j
    38.980003   +92.98576j     97.73398    +63.70668j
  -287.45038   -158.26791j    321.8197    +151.30878j
  -136.15845   -139.39746j    -22.614635   +88.557274j
   -75.58975    +16.577293j    75.36231    -14.467594j
    75.512596  -103.24147j    -65.76552    +48.333424j
    51.243362   +15.50803j    -77.67269    +78.130295j
    36.67746    -47.848145j    -0.8291626  -37.583656j
    54.28049    +15.14283j    -69.24906    +49.755505j
    17.982956  -137.15686j      8.250641  +204.92694j
   -84.98456   -216.10516j    133.65607   +204.3157j
  -108.657455  -182.85588j    156.60089   +116.13028j
  -201.24722    -19.4309j      75.85985    -19.267372j
    83.98462    -15.202744j   -22.855158   +40.50888j
  -153.69621    +14.0824585j  111.61706    -98.27

In [ ]:
spectrogram3 = tf.signal.stft(X_train_f[0], frame_length=256, frame_step=128)

print(X_train[0].shape[0])
print(spectrogram3.shape)
print(spectrogram3)

512
(3, 129)
tf.Tensor(
[[ 1.74082336e+02+0.00000000e+00j -5.21395264e+01+2.76592827e+01j
  -4.34999504e+01-2.37018661e+01j -1.23779287e+01-2.63398438e+01j
   5.74418411e+01+8.28758926e+01j -6.69484482e+01+1.77158051e+01j
   9.53649139e+00-1.50775620e+02j  1.78270721e+00+1.61253799e+02j
   4.01377487e+01-1.23790428e+02j -5.49727325e+01+5.69528275e+01j
   6.64112854e+01+1.20195312e+01j -3.46522522e+00-2.41517963e+01j
  -4.37877808e+01-3.98868561e+00j -4.19033051e+00+3.10579948e+01j
   3.83658791e+01-3.84720688e+01j -5.40863228e+01+2.80017929e+01j
   7.43911819e+01-4.49812546e+01j -5.51836395e+01+5.61540375e+01j
  -1.31282272e+01-6.60320587e+01j  7.18037415e+01+7.05327682e+01j
  -3.80010681e+01+1.03380356e+01j -2.43537636e+01-1.04537605e+02j
   1.85668221e+01+1.07560631e+02j -1.18270845e+01-4.89343185e+01j
   2.64039574e+01+1.02416840e+01j -4.06037636e+01-3.15140495e+01j
  -9.18457031e-01+3.53185959e+01j  8.28888245e+01-7.93958282e+00j
  -4.07435532e+01+4.77306747e+00j -5.17303925e+01-5.

In [ ]:
spectrogram4 = tf.signal.stft(X_train_f[0], frame_length=16, frame_step=8)

print(X_train[0].shape[0])
print(spectrogram4.shape)
print(spectrogram4)

512
(63, 9)
tf.Tensor(
[[-3.23367691e+00 +0.j         -1.27394753e+01 -8.01105j
  -1.90715194e+00+21.408302j    2.83625851e+01-11.801665j
  -3.00386887e+01 -6.341336j    3.19647865e+01+12.742952j
  -7.19284821e+00 -1.9863195j   2.81210518e+00-15.666433j
  -1.92889519e+01 +0.j        ]
 [-2.55710068e+01 +0.j          1.73538933e+01+20.984705j
  -7.01100111e+00-35.731136j    6.21483135e+00+35.52117j
  -6.94947720e+00-20.657982j    2.70912361e+00-10.461316j
   2.46100140e+00+34.42635j    -2.34778481e+01-31.297787j
   4.29699631e+01 +0.j        ]
 [ 9.47807217e+00 +0.j         -3.62726784e+00+20.643728j
  -3.71865940e+00-25.960903j    1.45754738e+01+12.316481j
  -1.00878153e+01 -3.4704094j  -1.99998436e+01 +1.7234373j
   2.23686581e+01-19.867863j   -8.14836407e+00+42.750687j
   7.79756069e+00 +0.j        ]
 [ 1.66481056e+01 +0.j         -8.02347565e+00 +2.7478733j
  -2.41150737e-01-17.021683j    1.10065107e+01 +6.748645j
  -2.37351379e+01 +7.3490047j   1.37887688e+01 +6.6751933j
  -3.58850

# Model Architecture <br>
* The querys is what you're trying to find.
* The keys what sort of information the dictionary has.
* The value is that information.


<b>fft_length and frame_length</b>
* The number of frequency bins depends on the FFT size (fft_length), which is often set to the next power of 2 that is greater than or equal to frame_length
* example, if frame_lenght = 39, Next power of 2 greater than 39 is 64. Setting fft_length = 64 would allow the FFT algorithm to work more efficiently by padding the input to a length of 64.
* or choose frame_lenght any of 2, 4, 8, 16, 32, 64, ... <br>

<b>frame_step</b>
* To choose frame_step, 50% Overlap (a common choice): To achieve 50% overlap, set frame_step to half the frame_length: frame_step = frame_length/2
* example, if frame_length=8, then frame_step=8/2 = 4. This will give you overlapping frames, which improves frequency resolution.<br>

<b>Frequency_bins v/s time_bins</b>
*  have more Frequency_bins if identifying a long-term uptrend or downtrend based on weekly or monthly stock price data, or detecting cyclic behavior like seasonality.
* have more time_bins if Detecting intraday trends or anomalies (like a sudden price surge due to a news release).

In [ ]:
# Hyperparameters for Fourier Transform
sample_signal_len = X_train.shape[1] # Length of the input time series
sample_frame_length = 8 # window size for Fourier Transform
sample_frame_step = 4 # Hop size for Fourier Transform
sample_fft_length = sample_frame_length

# Hyperparameters for Attention Layer and DNN Layer
sample_num_layers = 2 # number of TransformerEncoderLayer layers (Original paper = 6)
sample_num_heads = 2 # number of self-attention heads in the MultiheadAttention layer (Original paper = 8)
sample_dropout_rate = 0.2 # Dropout rate

In [ ]:
# The STFT output shape can be defined as:
# Output Shape = (batch_size, 𝐹, 𝑇)

# Number of Time Frames (T)
sample_time_bins = int(((sample_signal_len - sample_fft_length)/sample_frame_step )+1 )
print('Time Bins =',sample_time_bins)

# Number of Frequency Bins (F)
sample_frequency_bins = int(tf.math.floor((sample_fft_length/2) +1))
print('Frequency Bins =', sample_frequency_bins)

print('output_shape=[batch_size, time_bins, frequency_bins]')
print('Output shape = (batch_size,',sample_time_bins,',',sample_frequency_bins,')')

Time Bins = 31
Frequency Bins = 5
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 31 , 5 )


## The Fourier Transform layer

The output is a 2D array of shape (n_freqs, n_times), where n_freqs is the number of frequency bins (same as the length of f), and n_times is the number of time segments (same as the length of t).
* Rows of Zxx correspond to different frequencies (i.e., values in the f array).
* Columns of Zxx correspond to different time segments (i.e., values in the t array).
* The magnitude of the complex numbers in Zxx (np.abs(Zxx)) represents the strength or amplitude of each frequency component at that time segment.
* The phase of the complex numbers in Zxx (np.angle(Zxx)) represents the phase information of the signal at that frequency and time.
*  A larger window (windows_size, hops) gives better frequency resolution but poorer time resolution. You may need to experiment with different values based on your data.

* tf.py_function allows TensorFlow to execute scipy_stft_fn, passing a tensor as input, converting it to a NumPy array internally, and returning a tensor that is compatible with TensorFlow.
* In the build method of the FourierTransform class, the shape of self.kernel should be a tuple, but it is set as a single value (self.windows_size). self.windows_size should be wrapped in parentheses to specify the shape correctly, assuming it's a 1D kernel.

In [ ]:
class FourierTransform(Layer):
  def __init__(self, signal_len, frame_length, frame_step):
    super(FourierTransform, self).__init__()
    self.signal_len = signal_len
    self.frame_length = frame_length
    self.frame_step = frame_step

  def build(self, input_shape):
    # Define weights
    self.kernel = self.add_weight(
        shape=(self.signal_len,), # the use of ',' after self.signal_len is a must, read the above mentioned comments point-2
        initializer=initializer_for_relu,
        trainable=False
    )

  def call(self, x):
    #window_gen = hann_window(self.windows_size)  # symmetric Gaussian window
    # Convert the waveform to a spectrogram via a STFT.
    spectrogram = tf.signal.stft(signals=x, frame_length=self.frame_length, frame_step=self.frame_step)
    magnitude_x = tf.math.abs(spectrogram)
    angle_x = tf.math.angle(spectrogram) # Disable it if using only magnitude as output
    magnitude_x = tf_round(magnitude_x, 4)
    angle_x = tf_round(angle_x, 4) # Disable it if using only magnitude as output
    return magnitude_x, angle_x

In [ ]:
# Create an Embedding Object
sft_layer = FourierTransform(signal_len=sample_signal_len, frame_length=sample_frame_length, frame_step=sample_frame_step)
sft_layer.build(X_train_f[:3].shape)
# Calling the function
out_sft_mag, out_sft_ang = sft_layer(X_train_f[:3])

In [ ]:
print(out_sft_mag.shape)

(3, 31, 5)


In [ ]:
X_train_f[0,:]

array([  2.75,  -4.4 ,  -0.95,  -1.3 ,  -1.5 ,   6.8 ,  -2.  ,   4.85,
         2.55,  -1.15,   8.95,   0.4 ,   1.45,  -5.05,   1.3 ,   6.7 ,
         2.  ,   4.15,  -5.8 ,  -4.75,   2.55,   0.35,  -1.25,   1.25,
        -1.4 ,   9.  ,  -1.7 ,  -3.2 ,   2.55,   0.7 , -11.65,   2.7 ,
         5.7 ,  -2.8 ,   1.25,  -3.65,   1.6 ,  -0.6 ,   1.4 ,   0.9 ,
         3.7 ,  -2.25,  -0.8 ,   0.3 ,  -2.7 ,   1.45,   1.6 ,   0.35,
         1.2 ,  -2.9 ,  -0.25,   1.1 ,   6.8 ,   2.5 ,  -2.9 ,   4.5 ,
        -0.5 ,  -5.5 ,   5.  ,  -2.15,  -0.85,  -2.  ,  -0.95,   2.95,
        -3.4 ,  -0.55,   0.15,  -3.25,  -4.95,  -3.4 ,  -0.35,  -1.45,
        -1.1 ,  -1.5 ,   4.15,  -4.35,  -1.45,   4.9 ,  10.3 ,   2.95,
         5.25,  -5.45,   3.5 ,   1.15,  -4.15,  -1.7 ,   3.15,   0.75,
        -1.95,   0.75,   4.  , -11.  ,   4.5 ,   0.5 ,   0.75,  -1.1 ,
         4.2 ,  -0.75,   2.45,   0.55,   9.9 ,   3.95,   1.4 ,   4.6 ,
        -2.95,  -0.5 ,  -3.25,   5.7 ,  -5.45,   3.2 ,  -4.45,  -2.15,
      

In [ ]:
# Inspecting the time-component when frequency-component = 3
out_sft_mag[0,2,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([3.4188, 6.5908, 6.8675, 5.0159, 9.7312], dtype=float32)>

In [ ]:
# Inspecting the time-component when frequency-component = 12
out_sft_mag[0,6,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 4.5455,  3.278 , 10.1575,  8.5061,  3.7045], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 3
out_sft_mag[0,:,1]

<tf.Tensor: shape=(31,), dtype=float32, numpy=
array([ 5.6092,  4.2095,  6.5908, 10.8186,  5.0964,  6.5199,  3.278 ,
        6.5213,  2.424 ,  4.1184,  2.6027,  2.9844,  8.8118,  2.8087,
        3.6865,  2.201 ,  8.7673,  2.9696,  9.2064,  9.3898,  4.1872,
        0.6022,  5.4316,  3.1601, 12.3933,  5.4261,  2.2882,  5.8877,
        8.5485,  4.6297, 10.2129], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 12
out_sft_mag[0,:,4]

<tf.Tensor: shape=(31,), dtype=float32, numpy=
array([ 7.7354,  1.8124,  9.7312,  8.0759,  1.9898, 11.2066,  3.7045,
        1.0174,  6.8309,  5.1962,  3.5155,  3.6781,  1.9179,  1.3523,
        5.0907,  5.0797,  0.919 ,  4.4529,  5.0932, 13.3979,  0.3328,
        2.2045, 15.8886,  7.2253,  7.4202,  8.7878, 16.5085,  0.0816,
        7.0677,  3.2022,  2.5184], dtype=float32)>

In [ ]:
print(out_sft_ang.shape)

(3, 31, 5)


In [ ]:
# Inspecting the time-component when frequency-component = 3
out_sft_ang[0,3,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 0.    , -2.6551,  0.4815,  0.3518,  3.1416], dtype=float32)>

In [ ]:
# Inspecting the time-component when frequency-component = 12
out_sft_ang[0,6,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 3.1416, -1.5188, -0.4319,  2.239 ,  3.1416], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 3
out_sft_ang[0,:,1]

<tf.Tensor: shape=(31,), dtype=float32, numpy=
array([ 1.8924,  2.8548, -1.2734, -2.6551,  1.44  ,  2.4645, -1.5188,
        2.7181,  1.2483, -2.3576,  0.9813, -1.3673,  3.1091, -1.1529,
       -0.5765, -0.8768, -0.0495,  0.7958,  1.4326, -1.9089, -0.3039,
       -2.0366,  1.2328,  2.7974,  2.9729, -1.2956, -1.6566,  0.8707,
       -2.2072,  3.0501, -1.6155], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 12
out_sft_ang[0,:,4]

<tf.Tensor: shape=(31,), dtype=float32, numpy=
array([3.1416, 0.    , 0.    , 3.1416, 0.    , 3.1416, 3.1416, 0.    ,
       0.    , 0.    , 3.1416, 0.    , 0.    , 0.    , 0.    , 3.1416,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 3.1416, 3.1416, 3.1416, 3.1416, 0.    , 0.    ],
      dtype=float32)>

## Custom Attention Block

In [ ]:
class BaseAttention(Layer):
  def __init__(self, frequency_bins, time_bins, **kwargs):
    super().__init__()
    self.mha = MultiHeadAttention(key_dim=frequency_bins,**kwargs)
    self.layernorm = LayerNormalization()
    self.add = Add()
    self.frequency_bins = frequency_bins
    self.time_bins = time_bins

  def build(self, input_shape):
    # Define weights
    self.kernel = self.add_weight(
        shape=(self.time_bins, self.frequency_bins),
        initializer=initializer_for_relu,
        trainable=True
    )

## self-attention layer - Magnitude

* tf.ensure_shape Updates the shape of a tensor and checks at runtime that the shape holds.
* When executed, this operation asserts that the input tensor x's shape is compatible with the shape argument.

In [ ]:
class SelfAttentionMagnitude(BaseAttention):
  def __init__(self, frequency_bins, time_bins, **kwargs):
      # Call the parent class (BaseAttention) constructor
      super().__init__(frequency_bins, time_bins, **kwargs)

  def call(self, magnitude):
    magnitude = tf.ensure_shape(magnitude, [None, self.time_bins, self.frequency_bins])
    attn_output = self.mha(
        query=magnitude,  # The querys is what you're trying to find.
        key=magnitude,  # The keys what sort of information the dictionary has.
        value=magnitude # The value is that information.
        )
    # Cache the attention scores for plotting later.
    #self.last_attn_scores = attn_scores

    x = self.add([magnitude, attn_output])
    x = self.layernorm(x)
    x = tf_round(x, 4)
    return x

In [ ]:
sample_lsa = SelfAttentionMagnitude(frequency_bins=sample_frequency_bins,
                                time_bins=sample_time_bins,
                                num_heads=sample_num_heads
                                )

In [ ]:
#sample_lsa.build(out_sft_mag.shape)
out_lsa_mag = sample_lsa(out_sft_mag)
print(out_sft_mag.shape)
print(out_lsa_mag.shape)

(3, 31, 5)
(3, 31, 5)


In [ ]:
out_lsa_mag[0,0,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([-1.5256, -0.8664,  0.8396,  0.8528,  0.6996], dtype=float32)>

## self-attention layer - Angle

* tf.ensure_shape Updates the shape of a tensor and checks at runtime that the shape holds.
* When executed, this operation asserts that the input tensor x's shape is compatible with the shape argument.

In [ ]:
class SelfAttentionAngle(BaseAttention):
  def __init__(self, frequency_bins, time_bins, **kwargs):
      # Call the parent class (BaseAttention) constructor
      super().__init__(frequency_bins, time_bins, **kwargs)

  def call(self, angle):
    angle = tf.ensure_shape(angle, [None, self.time_bins, self.frequency_bins])
    attn_output = self.mha(
        query=angle,  # The querys is what you're trying to find.
        key=angle,  # The keys what sort of information the dictionary has.
        value=angle # The value is that information.
        )
    # Cache the attention scores for plotting later.
    #self.last_attn_scores = attn_scores

    x = self.add([angle, attn_output])
    x = self.layernorm(x)
    x = tf_round(x, 4)
    return x

In [ ]:
sample_lsa_angle = SelfAttentionAngle(frequency_bins=sample_frequency_bins,
                                time_bins=sample_time_bins,
                                num_heads=sample_num_heads
                                #key_dim=sample_d_model
                                #kernel_initializer=initializer_for_relu
                                )

In [ ]:
#sample_lsa.build(out_sft_mag.shape)
out_lsa_ang = sample_lsa_angle(out_sft_ang)
print(out_sft_ang.shape)
print(out_lsa_ang.shape)

(3, 31, 5)
(3, 31, 5)


In [ ]:
out_lsa_ang[0,0,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([-0.1781,  0.552 , -1.6412, -0.1297,  1.3969], dtype=float32)>

## The global cross-attention layer

The similarity (or dot product) between the Query and each Key is computed to determine an attention score. These scores measure how relevant each item (Key) is to the current item (Query).

In [ ]:
class GlobalCrossAttention(BaseAttention):
  def __init__(self, frequency_bins, time_bins, **kwargs):
      # Call the parent class (BaseAttention) constructor
      super().__init__(frequency_bins, time_bins, **kwargs)

  def call(self, magnitude, angle):
    magnitude = tf.ensure_shape(angle, [None, self.time_bins, self.frequency_bins])
    angle = tf.ensure_shape(angle, [None, self.time_bins, self.frequency_bins])
    attn_output, attn_scores = self.mha(
        query=magnitude,  # The querys is what you're trying to find.
        key=angle,  # The keys what sort of information the dictionary has.
        value=angle, # The value is that information.
        return_attention_scores=True
        )

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([magnitude, attn_output])
    x = self.layernorm(x)
    x = tf_round(x, 4)
    return x

In [ ]:
sample_gca = GlobalCrossAttention(frequency_bins=sample_frequency_bins,
                                  time_bins=sample_time_bins,
                                  num_heads=sample_num_heads
                                  #key_dim=sample_d_model
                                  )
#sample_gca.build(out_lsa_ang.shape)

In [ ]:
out_gca_mag = sample_gca(magnitude=out_lsa_mag, angle=out_lsa_ang)
print(out_lsa_mag.shape)
print(out_gca_mag.shape)

(3, 31, 5)
(3, 31, 5)


In [ ]:
out_gca_mag[0,0,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([-0.6096,  0.3554, -1.4411,  0.1464,  1.5488], dtype=float32)>

## Feed Forward Network Layer

In [ ]:
class FeedForward(Layer):
  # dff - dence feed forward neurons
  # sft_len - output shape Fourier Transform
  def __init__(self, frequency_bins, dropout_rate=0.1):
    super().__init__()
    self.seq = Sequential([
      Dense(frequency_bins*3, activation='relu', kernel_initializer=initializer_for_relu),
      Dropout(dropout_rate),
      Dense(frequency_bins*2, activation='relu', kernel_initializer=initializer_for_relu),
      Dropout(dropout_rate),
      Dense(frequency_bins, activation='relu', kernel_initializer=initializer_for_relu),
      Dropout(dropout_rate)
    ])
    self.add = Add()
    self.layer_norm = LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x

In [ ]:
out_gca_mag.shape[2]

5

In [ ]:
sample_ffn = FeedForward(frequency_bins=sample_frequency_bins)
sample_ffn_out = sample_ffn(out_gca_mag)
print(out_gca_mag.shape)
print(sample_ffn_out.shape)

(3, 31, 5)
(3, 31, 5)


In [ ]:
sample_ffn_out[0,0,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.3073068 ,  0.6797909 , -1.4000016 , -0.47473735,  1.5022547 ],
      dtype=float32)>

## Complete Encoder Layer

In [ ]:
class EncoderLayer(Layer):
  def __init__(self,*, frequency_bins, time_bins, num_heads, dropout_rate=0.1):
    super().__init__()

    self.self_attention = SelfAttentionAngle(frequency_bins=frequency_bins,
                                                 time_bins=time_bins,
                                                 num_heads=num_heads,
                                                 dropout=dropout_rate
                                                 )

    self.ffn = FeedForward(frequency_bins=frequency_bins)

  def call(self, angle):
    angle = self.self_attention(angle)
    angle = self.ffn(angle)
    return angle

In [ ]:
sample_encoder_layer = EncoderLayer(frequency_bins=sample_frequency_bins,
                                    time_bins=sample_time_bins,
                                    num_heads=sample_num_heads)
#sample_encoder_layer.build(out_sft_ang.shape)

In [ ]:
sample_encoder_output = sample_encoder_layer(out_sft_ang, training=False)
# Print the shape.
print(out_sft_ang.shape)
print(sample_encoder_output.shape)

(3, 31, 5)
(3, 31, 5)


In [ ]:
class Encoder(Layer):
  def __init__(self, *, frequency_bins, time_bins, num_layers, num_heads, dropout_rate=0.1):
    super().__init__()

    self.num_layers=num_layers

    self.enc_layers = [ EncoderLayer(
        frequency_bins=frequency_bins,
        time_bins=time_bins,
        num_heads=num_heads,
        dropout_rate=dropout_rate
    ) for _ in range(num_layers)]

  def call(self, angle):
    for i in range(self.num_layers):
      angle = self.enc_layers[i](angle)

    #self.last_attn_scores = self.enc_layers[-1].last_attn_scores
    return angle

In [ ]:
# Testing the encoder

# Instantiate the encoder.
sample_encoder = Encoder(frequency_bins=sample_frequency_bins,
                         time_bins=sample_time_bins,
                         num_layers=sample_num_layers,
                         num_heads=sample_num_heads)
#sample_encoder.build(out_sft_ang.shape)

In [ ]:
sample_encoder_output = sample_encoder(out_sft_ang, training=False)
# Print the shape.
print(out_sft_ang.shape)
print(sample_encoder_output.shape)

(3, 31, 5)
(3, 31, 5)


In [ ]:
sample_encoder_output[0,0,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.06786245,  0.9124432 , -1.3849719 , -0.7538711 ,  1.2942622 ],
      dtype=float32)>

## Complete Decoder Layer

Each DecoderLayer containing a CausalSelfAttention, a CrossAttention, and a FeedForward layer:

In [ ]:
class DecoderLayer(Layer):
  def __init__(self, *, frequency_bins, time_bins, num_heads, dropout_rate=0.1):
    super().__init__()

    self.local_self_att = SelfAttentionAngle(
        frequency_bins=frequency_bins,
        time_bins=time_bins,
        num_heads=num_heads,
        dropout=dropout_rate
    )

    self.global_cross_att = GlobalCrossAttention(
        frequency_bins=frequency_bins,
        time_bins=time_bins,
        num_heads=num_heads,
        dropout=dropout_rate
    )

    self.ffn = FeedForward(frequency_bins)

  def call(self, magnitude, angle):
    self_att_out = self.local_self_att(magnitude)
    cross_att_out = self.global_cross_att(magnitude=self_att_out, angle=angle)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.global_cross_att.last_attn_scores

    fnn_out = self.ffn(cross_att_out)

    return fnn_out


In [ ]:
# Testing decoder layer
sample_decoder_layer = DecoderLayer(frequency_bins=sample_frequency_bins,
                                    time_bins=sample_time_bins,
                                    num_heads=sample_num_heads)
#sample_decoder_layer.build(out_sft_mag.shape)
sample_decoder_layer_output = sample_decoder_layer(magnitude=out_sft_mag, angle=sample_encoder_output)

In [ ]:
print(out_sft_mag.shape)
print(sample_decoder_layer_output.shape)

(3, 31, 5)
(3, 31, 5)


In [ ]:
sample_decoder_layer_output[0,0,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 0.16463208,  0.45125467, -1.4211748 , -0.6982664 ,  1.5035543 ],
      dtype=float32)>

In [ ]:
class Decoder(Layer):
  def __init__(self, *, frequency_bins, time_bins, num_heads, num_layers, dropout_rate=0.1):
    super().__init__()

    self.num_layers=num_layers

    self.decoder_layer = [ DecoderLayer(
        frequency_bins=frequency_bins,
        time_bins=time_bins,
        num_heads=num_heads,
        dropout_rate=dropout_rate
    ) for _ in range(num_layers)]

  def call(self, magnitude, angle):
    for i in range(self.num_layers):
      magnitude  = self.decoder_layer[i](magnitude, angle)

    self.last_attn_scores = self.decoder_layer[-1].last_attn_scores

    return magnitude

In [ ]:
# Test the decoder

# Instantiate the decoder.
sample_decoder = Decoder(frequency_bins=sample_frequency_bins,
                         time_bins=sample_time_bins,
                         num_layers=sample_num_layers,
                         num_heads=sample_num_heads)
#sample_decoder.build(out_sft_mag.shape)
output = sample_decoder(magnitude=out_sft_mag, angle=sample_encoder_output)

In [ ]:
output[0,0,:]

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 0.05843002,  1.1890314 , -1.4372439 , -0.7705797 ,  0.96036226],
      dtype=float32)>

In [ ]:
print(out_sft_mag.shape)
print(sample_encoder_output.shape)
print(output.shape)

(3, 31, 5)
(3, 31, 5)
(3, 31, 5)


## Transformer

In [ ]:
class Transformer(Model):
  def __init__(self, *, signal_len, frame_length, frame_step,
               frequency_bins, time_bins,
               num_heads, num_layers, dropout_rate=0.1):
    super().__init__()

    self.sft_layer = FourierTransform(signal_len=signal_len,
                                      frame_length=frame_length,
                                      frame_step=frame_step)

    self.encoder = Encoder(frequency_bins=frequency_bins,
                           time_bins=time_bins,
                           num_layers=num_layers,
                           num_heads=num_heads,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(frequency_bins=frequency_bins,
                           time_bins=time_bins,
                           num_layers=num_layers,
                           num_heads=num_heads,
                           dropout_rate=dropout_rate)

    self.flatten_layer = Flatten()

    self.final_layer = Dense(1) # STUPID !!!!!! - using activation='relu' will limit the output between 0 and infinity, it won't give -ve outputs

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    #self.sft_layer.build(inputs.shape)
    magnitude_x, angle_x = self.sft_layer(inputs)

    #self.encoder.build(angle_x.shape)
    enc_out = self.encoder(angle_x)  # (batch_size, frequency_bins, time_bins)

    #self.decoder.build(magnitude_x.shape)
    dec_out = self.decoder(magnitude_x, enc_out)  # (batch_size, frequency_bins, time_bins)

    # Final linear layer output.
    dec_out = self.flatten_layer(dec_out) # (batch_size, frequency_bins * time_bins)
    logits = self.final_layer(dec_out)  # (batch_size, target_len)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

# Model 1

## Model building

In [ ]:
#Define a model with Transformer layer
tf.keras.backend.clear_session()
#model = tf.keras.Sequential()

In [ ]:
# Hyperparameters for Fourier Transform
signal_len = X_train.shape[1] # Length of the input time series
frame_length = 64 # window size for Fourier Transform
frame_step = 32 # Hop size for Fourier Transform
fft_length = frame_length

# Hyperparameters for Attention Layer and DNN Layer
num_layers = 2 # number of TransformerEncoderLayer layers (Original paper = 6)
num_heads = 1 # number of self-attention heads in the MultiheadAttention layer (Original paper = 8)
dropout_rate = 0.2 # Dropout rate

In [ ]:
# The STFT output shape can be defined as:
# Output Shape = (batch_size, 𝐹, 𝑇)

# Number of Time Frames (T)
time_bins = int(((signal_len - fft_length)/frame_step )+1 )
print('Time Bins =',time_bins)

# Number of Frequency Bins (F)
frequency_bins = int(tf.math.floor((fft_length/2) +1))
print('Frequency Bins =', frequency_bins)

print('output_shape=[batch_size, time_bins, frequency_bins]')
print('Output shape = (batch_size,',time_bins,',',frequency_bins,')')

Time Bins = 3
Frequency Bins = 33
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 3 , 33 )


In [ ]:
transformer = Transformer(signal_len=signal_len,
                          frame_length=frame_length,
                          frame_step=frame_step,
                          frequency_bins=frequency_bins,
                          time_bins=time_bins,
                          num_layers=num_layers,
                          num_heads=num_heads,
                          dropout_rate=dropout_rate)

In [ ]:
transformer.build(X_train_f.shape)

In [ ]:
transformer_output = transformer(X_train_f[:3])

print(X_train_f[:3].shape)
print(transformer_output.shape)

(3, 128)
(3, 1)


In [ ]:
attn_scores = transformer.decoder.decoder_layer[-1].last_attn_scores
print(attn_scores.shape)

(3, 1, 3, 3)


In [ ]:
transformer.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ fourier_transform (FourierTransform) │ ?                           │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder (Encoder)                    │ ?                           │          18,348 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder (Decoder)                    │ ?                           │          27,654 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (3, 99)                     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (3, 1)                      │             100 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 46,230 (180.59 KB)

 Trainable params: 46,102 (180.09 KB)

 Non-trainable params: 128 (512.00 B)

## Training

In [ ]:
#learning_rate = CustomSchedule(d_model)
optimizer = Adam(0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-8)

In [ ]:
transformer.compile(
    loss='mean_absolute_error',#'mean_absolute_error',
    optimizer=optimizer,
    metrics=['mean_squared_error'])# 'mean_squared_error'

In [ ]:
path_model = '/content/drive/MyDrive/hobby_project/Models'
checkpoint_filepath = (path_model+'/1SF_nsepy_Model_inp512_out1_fourierTransform.weights.h5')
print(checkpoint_filepath)

/content/drive/MyDrive/hobby_project/Models/1SF_nsepy_Model_inp512_out1_fourierTransform.weights.h5


In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    #monitor='loss',
    mode='min',
    save_best_only=True)

early_stopping = EarlyStopping(monitor="val_loss",
                               #monitor="loss",
                               patience=25, mode="min",
                               restore_best_weights=True)

In [ ]:
transformer.fit(X_train_f,y_train,
                epochs=100, batch_size=32,
                validation_data=[X_val_f,y_val],
                callbacks=[model_checkpoint_callback, early_stopping])

Epoch 1/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 16s 170ms/step - loss: 3.2896 - mean_squared_error: 24.9134 - val_loss: 3.3477 - val_mean_squared_error: 24.0071
Epoch 2/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 3.1975 - mean_squared_error: 23.2319 - val_loss: 3.2647 - val_mean_squared_error: 23.3308
Epoch 3/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.0418 - mean_squared_error: 25.2945 - val_loss: 3.2140 - val_mean_squared_error: 23.0161
Epoch 4/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.9668 - mean_squared_error: 21.9431 - val_loss: 3.1694 - val_mean_squared_error: 22.6407
Epoch 5/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.8454 - mean_squared_error: 19.8408 - val_loss: 3.1308 - val_mean_squared_error: 22.4587
Epoch 6/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.7509 - mean_squared_error: 19.3416 - val_loss: 3.1013 - val_mean_squared_error: 22.2271
Epoch 7/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.7532 - mean_squared_error: 20.5687 

In [ ]:
transformer.fit(X_train_f,y_train,
                initial_epoch=100, epochs=200, batch_size=32,
                validation_data=[X_val_f,y_val],
                callbacks=[model_checkpoint_callback, early_stopping])

Epoch 101/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.5468 - mean_squared_error: 5.0224 - val_loss: 1.3722 - val_mean_squared_error: 10.0680
Epoch 102/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5842 - mean_squared_error: 5.1002 - val_loss: 1.3862 - val_mean_squared_error: 10.1100
Epoch 103/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.5119 - mean_squared_error: 4.8308 - val_loss: 1.3638 - val_mean_squared_error: 9.9926
Epoch 104/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.5606 - mean_squared_error: 5.9100 - val_loss: 1.3603 - val_mean_squared_error: 10.0050
Epoch 105/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5293 - mean_squared_error: 3.9253 - val_loss: 1.3704 - val_mean_squared_error: 9.9032
Epoch 106/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4530 - mean_squared_error: 3.0891 - val_loss: 1.3639 - val_mean_squared_error: 9.9513
Epoch 107/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.5635 - mean_squared_error: 6.0277 - va

In [ ]:
transformer.fit(X_train_f,y_train,
                initial_epoch=200, epochs=500, batch_size=32,
                validation_data=[X_val_f,y_val],
                callbacks=[model_checkpoint_callback, early_stopping])

Epoch 201/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3506 - mean_squared_error: 2.4785 - val_loss: 1.1833 - val_mean_squared_error: 7.9003
Epoch 202/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3582 - mean_squared_error: 2.8334 - val_loss: 1.1943 - val_mean_squared_error: 7.9155
Epoch 203/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3084 - mean_squared_error: 1.3489 - val_loss: 1.1867 - val_mean_squared_error: 8.0290
Epoch 204/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3381 - mean_squared_error: 1.9697 - val_loss: 1.1885 - val_mean_squared_error: 7.9120
Epoch 205/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4086 - mean_squared_error: 3.4534 - val_loss: 1.2105 - val_mean_squared_error: 7.9637
Epoch 206/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3257 - mean_squared_error: 2.0609 - val_loss: 1.1978 - val_mean_squared_error: 7.8911
Epoch 207/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3324 - mean_squared_error: 2.4540 - val_los

## Prediction - from Transformer

In [ ]:
predictions = transformer((X_test_f), training=False)

In [ ]:
predictions.shape

TensorShape([240, 1])

In [ ]:
predictions[0]

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-1.3341217], dtype=float32)>

In [ ]:
i = 12
print('Predictions = \n',predictions[i])
print('Actuals = \n',y_test[i])

Predictions = 
 tf.Tensor([-0.34423143], shape=(1,), dtype=float32)
Actuals = 
 -0.25


### Validating the results

In [ ]:
y_pred = predictions.numpy().reshape(-1)
y_test = y_test.reshape(-1)
print(y_pred.shape)
print(y_test.shape)

(240,)
(240,)


In [ ]:
score_mae = mean_absolute_error(y_test, y_pred)
print("The Mean Absolute Error of our Model is {}".format(round(score_mae, 2)))
score_rmse = mean_squared_error(y_test, y_pred)
print("The Root Mean Squared Error of our Model is {}".format(round(score_rmse, 2)))

The Mean Absolute Error of our Model is 1.13
The Root Mean Squared Error of our Model is 7.41


In [ ]:
fig = go.Figure(data=[go.Ohlc(x=np.arange(0,1000,1),
                    open=y_pred,
                    high=y_pred,
                    low=y_test,
                    close=y_test
                              )])
fig.show()

In [ ]:
'''
fig = go.Figure(data=[go.Ohlc(x=np.arange(0,1000,1),
                    open=y_pred,
                    high=y_pred,
                    low=y_test,
                    close=y_test
                              )])
fig.show()
'''

'\nfig = go.Figure(data=[go.Ohlc(x=np.arange(0,1000,1),\n                    open=y_pred,\n                    high=y_pred,\n                    low=y_test,\n                    close=y_test\n                              )])\nfig.show()\n'

 ## Prediction - from Saved model

In [ ]:
# Save the model - not required - using different save method

#model_path = '/content/drive/MyDrive/hobby_project/Models/forecaster_future1_pred_finetune/'
#tf.saved_model.save(osf_tf_function, export_dir=model_path)

In [ ]:
# Load the model
model_path = '/content/drive/MyDrive/hobby_project/Models/1SF_nsepy_Model_inp512_out1_fourierTransform/'
pre_trained_forecaster = tf.saved_model.load(model_path)

In [ ]:
print(checkpoint_filepath)

/content/drive/MyDrive/hobby_project/Models/1SF_nsepy_Model_inp39_out1_vocab90.weights.weights.h5


In [ ]:
transformer.load_weights(checkpoint_filepath)

### Prediction from Pre-trained loaded transformer

In [ ]:
predictions_reconstructed_model = transformer((c_test_pad[:1000],X_test_pad[:1000]), training=False)
y_test_redused = y_test_pad[:1000]

In [ ]:
predictions_val_reconstructed_model = np.argmax(predictions_reconstructed_model, axis=2)
predictions_val_reconstructed_model.shape

(1000, 41)

In [ ]:
i = 1
print('Predictions = \n',predictions_val_reconstructed_model[i])
print('Actuals = \n',y_test_redused[i])

Predictions = 
 [39 38 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39
 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39]
Actuals = 
 [53  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 89]


### Compare saved model with Actuals

In [ ]:
y_reconstructed_model = predictions_val_reconstructed_model[:,1]

In [ ]:
print(y_reconstructed_model.shape)
print(y_test[:1000].shape)

accuracy = accuracy_score(y_test[:1000],y_reconstructed_model)
print(f"Accuracy: {accuracy:.2f}")

(1000,)
(1000, 1)
Accuracy: 0.02


In [ ]:
fig = go.Figure(data=[go.Ohlc(x=np.arange(0,1000,1),
                    open=y_reconstructed_model,
                    high=y_reconstructed_model,
                    low=y_test[:1000].reshape(-1),
                    close=y_test[:1000].reshape(-1)
                              )])
fig.show()

### Exporting the outputs to csv

In [ ]:
data_df = pd.read_csv(path+'/OSF_results.csv')
data_df.columns

Index(['Actuals', 'OSF_Model_1', 'OSF_2'], dtype='object')

In [ ]:
# Uncomment only if you wish to compare the data in excel sheet
data_df = pd.read_csv(path+'/OSF_results.csv')
#data_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

# Using DataFrame.insert() to add a column
#data_df.insert(2, "OSF_2", y_reconstructed_model, True)
data_df.insert(2, "OSF_2_better", y_pred, True)

data_df

,Actuals,OSF_Model_1,OSF_2_better,OSF_2
0,542,542,542,542
1,509,509,509,509
2,722,562,586,588
3,517,517,517,517
4,525,525,525,525
...,...,...,...,...
2518,520,520,520,520
2519,500,500,500,500
2520,542,542,542,542
2521,523,523,523,523


In [ ]:
data_df.to_csv(path+'/OSF_results.csv',index=False)

# Failed Models

# Success Models

In [ ]:
# 1st Best model
# Inputs
'''
X Train shape (1620, 128)
Context Train shape (1620, 1)
Y Train shape (1620, 1)
X Validation shape (540, 128)
Context Validation shape (540, 1)
Y Validation shape (540, 1)
X Test shape (240, 128)
Context Test shape (240, 1)
Y Test shape (240, 1)
'''
# Hyperparameters
'''
# Hyperparameters for Fourier Transform
signal_len = X_train.shape[1] # Length of the input time series
frame_length = 64 # window size for Fourier Transform
frame_step = 32 # Hop size for Fourier Transform
fft_length = frame_length

# Hyperparameters for Attention Layer and DNN Layer
num_layers = 2 # number of TransformerEncoderLayer layers (Original paper = 6)
num_heads = 1 # number of self-attention heads in the MultiheadAttention layer (Original paper = 8)
dropout_rate = 0.2 # Dropout rate

Time Bins = 3
Frequency Bins = 33
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 3 , 33 )
'''
# Weights
'''
Model: "transformer"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ fourier_transform (FourierTransform) │ ?                           │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder (Encoder)                    │ ?                           │          18,348 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder (Decoder)                    │ ?                           │          27,654 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (3, 99)                     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (3, 1)                      │             100 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
 Total params: 46,230 (180.59 KB)
 Trainable params: 46,102 (180.09 KB)
 Non-trainable params: 128 (512.00 B)
'''

# Output
'''
Test
The Mean Absolute Error of our Model is 1.13
The Root Mean Squared Error of our Model is 7.41